In [95]:
import pandas as pd
from glob import glob
from tqdm import tqdm

In [106]:
d = "/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/csv/*.csv"
fnames = glob(d)
print(f"Total csv in dir: {len(fnames)}")
total_in_each_tic = {}
list_to_redo_fetch = []
all_df = None
tickers = []
for fname in fnames:
    dfi = pd.read_csv(fname)
    tic = fname.split("/")[-1][:4]
    total = len(dfi)
    total_in_each_tic[tic] = total
    if total < 3627:
        list_to_redo_fetch.append(tic)
    else:
        if all_df is not None:
            all_df = pd.concat([all_df, dfi], ignore_index=True)
            tickers.append(tic)
        else:
            all_df = dfi
            tickers.append(tic)

unique_dates = sorted(all_df["date"].unique())
date2day = {dt:i for i,dt in enumerate(unique_dates, start=1)}
all_df["day"] = all_df["date"].map(date2day)
all_df = all_df.sort_values(by='day', ascending=True)
        
# total_data_in_each_tic
list_to_repeat_fetch_fname = "tuntun_scripts/tics_to_repeat_fetch.txt"
with open(list_to_repeat_fetch_fname, "w+") as f:       
    x = "\n".join(list_to_redo_fetch)
    f.write(x)
    
# print(tickers[:10])
print(f"Total tickers with 3627 days: {len(tickers)}")
print(f"Total tickers to redo fetch: {len(list_to_redo_fetch)}")

import pandas as pd
processed_data_fname = f"/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/processed_data/{len(tickers)}_tickers_daily.csv"
all_df.to_csv(processed_data_fname, index=False)

Total csv in dir: 937
Total tickers with 3627 days: 355
Total tickers to redo fetch: 582


In [140]:
processed_data_fname = f"/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/processed_data/355_tickers_daily_with_dummy_rows.csv"
all_df = pd.read_csv(processed_data_fname)
all_df

,date,open,high,low,close,volume,tic,day
0,2010-01-04,21486.79,22725.50,21486.79,22725.50,1006978.0,AALI,1
1,2010-01-05,22820.78,23106.64,22630.21,23058.99,1761556.0,AALI,2
2,2010-01-06,23154.28,23249.56,22916.07,22963.71,1070472.0,AALI,3
3,2010-01-07,23058.99,23916.56,23058.99,23868.92,2431650.0,AALI,4
4,2010-01-08,23821.28,24202.42,23773.63,24059.49,949256.0,AALI,5
...,...,...,...,...,...,...,...,...
1288645,2024-11-20,98.00,105.00,94.00,98.00,0.0,ZBRA,3626
1288646,2024-11-21,98.00,105.00,94.00,98.00,0.0,ZBRA,3627
1288647,2024-11-22,78.00,79.00,74.00,78.00,1205900.0,ZBRA,3628
1288648,2024-11-25,78.00,105.00,77.00,78.00,96419600.0,ZBRA,3629


In [155]:
len(all_df["tic"].unique())

355

In [163]:
import pandas as pd

# Assuming the DataFrame is already loaded as 'df'
# nan_volume_count = all_df[all_df['volume'].isna()].groupby('tic').size()
nan_volume_count = all_df[all_df['volume'] == 0].groupby('tic').size()

# print("Number of rows with NaN volume for each ticker:")
# print(nan_volume_count)

# Optional: If you want to see tickers with NaN volumes sorted from most to least
print("\nSorted from most NaN volumes to least:")
tickers = sorted(nan_volume_count.sort_values(ascending=True)[:100].keys().tolist())
# print(tickers)
all_df = all_df[all_df["tic"].isin(tickers)]
print(all_df)


Sorted from most NaN volumes to least:
               date   open   high    low  close      volume   tic   day
10890    2010-01-04  151.0  151.0  147.0  148.0   1380000.0  ACES     1
10891    2010-01-05  149.0  149.0  147.0  149.0   2630000.0  ACES     2
10892    2010-01-06  149.0  150.0  149.0  150.0    495000.0  ACES     3
10893    2010-01-07  149.0  149.0  147.0  149.0    995000.0  ACES     4
10894    2010-01-08  149.0  149.0  148.0  149.0    405000.0  ACES     5
...             ...    ...    ...    ...    ...         ...   ...   ...
1274125  2024-11-20  386.0  396.0  382.0  386.0     23600.0  WIKA  3626
1274126  2024-11-21  380.0  386.0  380.0  380.0       600.0  WIKA  3627
1274127  2024-11-22  322.0  328.0  314.0  322.0  11545200.0  WIKA  3628
1274128  2024-11-25  314.0  328.0  314.0  314.0  23877400.0  WIKA  3629
1274129  2024-11-26  324.0  326.0  304.0  324.0  18459500.0  WIKA  3630

[363000 rows x 8 columns]


In [162]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [148]:
# all_df = all_df[(0 <= all_df["day"]) & (all_df["day"] <= 3630)]
# all_df

In [165]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
)
# INDICATORS = [""]
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list=INDICATORS,
                     use_turbulence=True,
                     user_defined_feature=False)

processed = fe.preprocess_data(all_df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

BEFORE CLEAN:                date   open   high    low  close      volume   tic   day
10890    2010-01-04  151.0  151.0  147.0  148.0   1380000.0  ACES     1
10891    2010-01-05  149.0  149.0  147.0  149.0   2630000.0  ACES     2
10892    2010-01-06  149.0  150.0  149.0  150.0    495000.0  ACES     3
10893    2010-01-07  149.0  149.0  147.0  149.0    995000.0  ACES     4
10894    2010-01-08  149.0  149.0  148.0  149.0    405000.0  ACES     5
...             ...    ...    ...    ...    ...         ...   ...   ...
1274125  2024-11-20  386.0  396.0  382.0  386.0     23600.0  WIKA  3626
1274126  2024-11-21  380.0  386.0  380.0  380.0       600.0  WIKA  3627
1274127  2024-11-22  322.0  328.0  314.0  322.0  11545200.0  WIKA  3628
1274128  2024-11-25  314.0  328.0  314.0  314.0  23877400.0  WIKA  3629
1274129  2024-11-26  324.0  326.0  304.0  324.0  18459500.0  WIKA  3630

[363000 rows x 8 columns]

AFTER CLEAN:             date     open     high      low    close      volume   tic   day
0   

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 14.07it/s]



boll_ub


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 14.08it/s]



boll_lb


...


close_30_sma


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 14.24it/s]



close_60_sma


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 14.31it/s]


Successfully added technical indicators
Successfully added turbulence index


In [166]:
features_csv = "/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/processed_data/100_tickers_with_features.csv"
# processed.to_csv(features_csv, index=False)

In [167]:
fdf = pd.read_csv(features_csv)
fdf

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2010-01-04,151.00,151.00,147.00,148.00,1380000.0,ACES,1,0.000000,149.914214,147.085786,100.000000,-66.666667,100.000000,148.000000,148.000000,0.000000
1,2010-01-04,630.00,630.00,620.00,630.00,162000.0,ADES,1,0.000000,149.914214,147.085786,100.000000,-66.666667,100.000000,630.000000,630.000000,0.000000
2,2010-01-04,347.87,347.87,343.63,347.87,760195.0,ADHI,1,0.000000,149.914214,147.085786,100.000000,-66.666667,100.000000,347.870000,347.870000,0.000000
3,2010-01-04,6800.00,6800.00,6800.00,6800.00,17500.0,ADMF,1,0.000000,149.914214,147.085786,100.000000,-66.666667,100.000000,6800.000000,6800.000000,0.000000
4,2010-01-04,134.00,142.00,134.00,142.00,20006500.0,ADMG,1,0.000000,149.914214,147.085786,100.000000,-66.666667,100.000000,142.000000,142.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362995,2024-11-26,2550.00,2560.00,2510.00,2550.00,1474900.0,TSPC,3630,-21.482045,2718.864030,2547.135970,47.421834,-168.905306,21.868859,2654.333333,2654.666667,249.067994
362996,2024-11-26,1670.00,1685.00,1665.00,1670.00,1319700.0,ULTJ,3630,-28.982196,1780.892823,1665.107177,39.086598,-101.521955,28.780787,1769.000000,1813.500000,249.067994
362997,2024-11-26,123.00,123.00,122.00,123.00,284200.0,UNSP,3630,5.733318,142.897927,103.302073,60.280480,39.975513,21.470936,115.433333,100.516667,249.067994
362998,2024-11-26,129.00,130.00,127.00,129.00,3067800.0,WEHA,3630,-3.676446,146.304369,124.495631,43.942857,-88.149445,25.055273,142.533333,140.466667,249.067994


In [111]:
total_in_each_tic["GTBO"]

3628

In [91]:
# tickers_str = "\n".join(tickers)
# tickers_fname = f"/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/{len(tickers)}_tickers_with_complete_data_from_01-01-2010_until_26-11-2024.txt"
# with open(tickers_fname, "w+") as f:
#     f.write(tickers_str)

In [44]:
all_df['volume'] = all_df['volume'].fillna(1)
all_df['close'] = all_df.apply(
    lambda row: row['open'] if pd.isna(row['close']) or row['close'] == 0 else row['close'],
    axis=1
)
all_df

,date,open,high,low,close,volume,tic,day
0,2010-01-04,598.49,598.49,598.49,598.49,1.0,FAST,4
1,2010-01-05,598.49,598.49,598.49,598.49,1.0,FAST,5
2,2010-01-06,598.49,598.49,598.49,598.49,1.0,FAST,6
3,2010-01-07,598.49,598.49,598.49,598.49,1.0,FAST,7
4,2010-01-08,690.57,690.57,690.57,690.57,1.0,FAST,8
...,...,...,...,...,...,...,...,...
493267,2024-11-20,60.00,65.00,60.00,60.00,1.0,DSFI,5438
493268,2024-11-21,60.00,65.00,60.00,60.00,1.0,DSFI,5439
493269,2024-11-22,58.00,58.00,54.00,58.00,1.0,DSFI,5440
493270,2024-11-25,56.00,57.00,54.00,56.00,1.0,DSFI,5443


In [127]:
processed_data_fname = f"/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/processed_data/355_tickers_daily_with_dummy_rows.csv"
all_df = pd.read_csv(processed_data_fname)
for i, row in tqdm(all_df.iterrows(), total=len(all_df)):
    # Check if any of the target columns are zero
    # if row['open'] == 0 or row['high'] == 0 or row['low'] == 0 or row['close'] == 0 or row['volume'] == 0:
        
    # If 'open' is zero, assign it the value of 'close', and vice versa
    # if row['volume'] == 0:
    #     all_df.at[i, 'volume'] = 1
    if row['open'] == 0:
        all_df.at[i, 'open'] = row['close']
    if row['close'] == 0:
        all_df.at[i, 'close'] = row['open']
    
    # Assign 'high' the maximum of 'open' and 'close' if it is zero
    if row['high'] == 0:
        all_df.at[i, 'high'] = max(row['open'], row['close'])
    
    # Assign 'low' the minimum of 'open' and 'close' if it is zero
    if row['low'] == 0:
        all_df.at[i, 'low'] = min(row['open'], row['close'])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1288650/1288650 [01:21<00:00, 15728.09it/s]


In [128]:
nan_count = all_df.isna().sum()

# Check for zero values in each column
zero_count = (all_df == 0).sum()

# Combine results into a summary DataFrame
summary = pd.DataFrame({
    'Column': all_df.columns,
    'NaN Count': nan_count.values,
    'Zero Count': zero_count.values
})

print("Detailed check for NaN and zero values in each column:")
print(summary)

Detailed check for NaN and zero values in each column:
   Column  NaN Count  Zero Count
0    date          0           0
1    open          0          56
2    high          0          56
3     low          0          56
4   close          0          56
5  volume          0      285612
6     tic          0           0
7     day          0           0


In [60]:
# Filter rows where any of the target columns ('open', 'high', 'low', 'close') have a zero value
rows_with_zeros = all_df[(all_df['open'] == 0) | 
                         (all_df['high'] == 0) | 
                         (all_df['low'] == 0) | 
                         (all_df['close'] == 0)]

# Print the rows with zero values
print("Rows with zero values in 'open', 'high', 'low', or 'close':")
print(rows_with_zeros)

Rows with zero values in 'open', 'high', 'low', or 'close':
Empty DataFrame
Columns: [date, open, high, low, close, volume, tic, day]
Index: []


In [51]:
def get_prev_open_market_value(all_df, zero_day, tic):
    # TODO: check the previous open market day from zero_day variable in 'day' column
    # WRONG: prev_open_day = zero_day - 1 # this is wrong because yesterday the market could be closed
    # i think first, we need to 
    # 1. tic_df = all_df[all_df[tic] == tic]
    # 2. tic_df = sort by 'day' column, ascending
    # 3. zero_day_df_index = tic_df[zero_day]
    # 4. prev_open_market_day_df_index = tic_df[zero_day_df_index-1]
    # 5. return tic_df[prev_open_market_day_df_index] as a dictionary

# TODO:
zero_tics = rows_with_zeros["tic"].unique()
for i, row in rows_with_zeros.iterrows():
    prev_row = get_prev_open_market_value(all_df, row["day"], row["tic"])
    # replace all_df[zero_day_df_index] with prev_row

SyntaxError: incomplete input (73672376.py, line 2)

In [58]:
def get_prev_open_market_value(all_df, zero_day, tic):
    """
    Get the previous open market day's data for a specific ticker.

    Parameters:
    - all_df (pd.DataFrame): The DataFrame containing stock data.
    - zero_day (int): The day index for which to find the previous open day.
    - tic (str): The ticker symbol.

    Returns:
    - dict: A dictionary containing the previous open day's row data.
    """
    # Filter the DataFrame for the specific ticker
    tic_df = all_df[all_df['tic'] == tic].sort_values(by='day')
    tic_df = tic_df.reset_index(drop=True)
    
    # Find the index of the current zero day in the sorted DataFrame
    zero_day_index = tic_df[tic_df['day'] == zero_day].index[0]
    
    print(f"ZERO DAY INDEX: {zero_day_index}")
    
    # Check for a previous day
    if zero_day_index == 0:
        print(f"IN FUNCTION. NO PREV DAY EXISTS")
        return None  # No previous day exists

    # Get the previous day's data
    prev_open_market_day_index = tic_df.index[zero_day_index - 1]
    return tic_df.loc[prev_open_market_day_index].to_dict()

# Iterate over rows with zeros and replace them
for i, row in rows_with_zeros.iterrows():
    # Get the previous market day data
    prev_row = get_prev_open_market_value(all_df, row['day'], row['tic'])

    # If no previous row exists, skip this entry
    if prev_row is None:
        print(f"NO PREV ROW FOR: {row['tic']}")
        continue

    # Replace the zero values with values from the previous market day
    for col in ['open', 'high', 'low', 'close']:
        if row[col] == 0:
            all_df.at[i, col] = prev_row[col]

ZERO DAY INDEX: 3107
ZERO DAY INDEX: 3164
ZERO DAY INDEX: 3107
ZERO DAY INDEX: 3107
ZERO DAY INDEX: 3107
ZERO DAY INDEX: 3163
ZERO DAY INDEX: 3107
ZERO DAY INDEX: 3107
ZERO DAY INDEX: 3107
ZERO DAY INDEX: 3201
ZERO DAY INDEX: 3107
ZERO DAY INDEX: 3107
ZERO DAY INDEX: 3107
ZERO DAY INDEX: 3107


In [73]:
all_df

,date,open,high,low,close,volume,tic,day
0,2010-01-04,598.49,598.49,598.49,598.49,1.0,FAST,1
1,2010-01-05,598.49,598.49,598.49,598.49,1.0,FAST,2
2,2010-01-06,598.49,598.49,598.49,598.49,1.0,FAST,3
3,2010-01-07,598.49,598.49,598.49,598.49,1.0,FAST,4
4,2010-01-08,690.57,690.57,690.57,690.57,1.0,FAST,5
...,...,...,...,...,...,...,...,...
493267,2024-11-20,60.00,65.00,60.00,60.00,1.0,DSFI,3623
493268,2024-11-21,60.00,65.00,60.00,60.00,1.0,DSFI,3624
493269,2024-11-22,58.00,58.00,54.00,58.00,1.0,DSFI,3625
493270,2024-11-25,56.00,57.00,54.00,56.00,1.0,DSFI,3626


In [123]:
from tqdm import tqdm
def add_dummy_row(all_df):
    unique_dates = sorted(all_df["date"].unique())
    unique_days = sorted(all_df["day"].unique())
    unique_tics = sorted(all_df["tic"].unique())
    new_all_df = pd.DataFrame()  # Initialize as empty DataFrame for concatenation

    for tic in tqdm(unique_tics, total=len(unique_tics)):
        tic_df = all_df[all_df["tic"] == tic]
        for day, date in zip(unique_days, unique_dates):
            
            # Create dummy row
            dummy_row = pd.DataFrame({
                "date": [date], 
                "open": [1], 
                "high": [1], 
                "low": [1], 
                "close": [1], 
                "volume": [1],
                "tic": [tic], 
                "day": [day]
            })
            
            # Check if this specific date is missing for the current ticker
            if date not in tic_df["date"].values:
                # If missing, add the dummy row
                tic_df = pd.concat([tic_df, dummy_row], ignore_index=True)
        
        tic_df = tic_df.sort_values(by='day')
        new_all_df = pd.concat([new_all_df, tic_df], ignore_index=True)
    new_all_df = new_all_df.reset_index(drop=True)
    return new_all_df
all_df = add_dummy_row(all_df)

In [124]:
tickers = all_df["tic"].unique()
processed_data_fname = f"/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/processed_data/{len(tickers)}_tickers_daily_with_dummy_rows.csv"
all_df.to_csv(processed_data_fname, index=False)

In [125]:
def validate_tic_dates(all_df):
    """
    Validates that each ticker (tic) in the DataFrame has the same unique dates.
    
    Steps:
    1. Counts the unique dates for each ticker and saves it in a dictionary.
    2. Asserts that all tickers have the same unique dates and the same total unique dates.

    Parameters:
    - all_df (pd.DataFrame): The DataFrame containing stock data with 'date' and 'tic' columns.

    Returns:
    - dict: A dictionary with tickers as keys and their total unique dates as values.
    - int: Total number of unique dates across the dataset.

    Raises:
    - AssertionError: If any ticker has a different set or total count of unique dates.
    """
    # Group the data by 'tic' and count unique dates for each tic
    unique_dates_per_tic = all_df.groupby("tic")["date"].nunique().to_dict()

    # Check that all tickers have the same total unique dates
    total_unique_dates = len(all_df["date"].unique())
    assert all(val == total_unique_dates for val in unique_dates_per_tic.values()), \
        "Not all tickers have the same number of unique dates!"

    # Check that all tickers have the same set of dates
    unique_date_sets = all_df.groupby("tic")["date"].apply(lambda x: set(x)).to_dict()

    # Ensure that all tickers have the same set of dates
    date_sets = list(unique_date_sets.values())
    assert all(date_set == date_sets[0] for date_set in date_sets), \
        "Not all tickers share the same set of dates!"

    # Return the results
    return unique_dates_per_tic, total_unique_dates

# Example usage:
new_total_for_each_tic, total_unique_dates = validate_tic_dates(all_df)
print("Unique dates per ticker:", new_total_for_each_tic)
print("Total unique dates in the dataset:", total_unique_dates)


Unique dates per ticker: {'AALI': 3630, 'ABBA': 3630, 'ABDA': 3630, 'ACES': 3630, 'ADES': 3630, 'ADHI': 3630, 'ADMF': 3630, 'ADMG': 3630, 'ADRO': 3630, 'AGRO': 3630, 'AHAP': 3630, 'AIMS': 3630, 'AISA': 3630, 'AKKU': 3630, 'AKPI': 3630, 'AKRA': 3630, 'AKSI': 3630, 'ALKA': 3630, 'ALMI': 3630, 'AMAG': 3630, 'AMFG': 3630, 'AMRT': 3630, 'ANTM': 3630, 'APIC': 3630, 'APLI': 3630, 'ARGO': 3630, 'ARNA': 3630, 'ARTA': 3630, 'ARTI': 3630, 'ASBI': 3630, 'ASDM': 3630, 'ASGR': 3630, 'ASII': 3630, 'ASJT': 3630, 'ASRI': 3630, 'ASRM': 3630, 'AUTO': 3630, 'BABP': 3630, 'BACA': 3630, 'BAPA': 3630, 'BATA': 3630, 'BAYU': 3630, 'BBCA': 3630, 'BBKP': 3630, 'BBLD': 3630, 'BBNI': 3630, 'BBRI': 3630, 'BBTN': 3630, 'BCAP': 3630, 'BCIC': 3630, 'BCIP': 3630, 'BDMN': 3630, 'BEKS': 3630, 'BFIN': 3630, 'BHIT': 3630, 'BIMA': 3630, 'BIPP': 3630, 'BISI': 3630, 'BKDP': 3630, 'BKSL': 3630, 'BKSW': 3630, 'BLTA': 3630, 'BMRI': 3630, 'BMSR': 3630, 'BMTR': 3630, 'BNBA': 3630, 'BNBR': 3630, 'BNGA': 3630, 'BNII': 3630, 'BNLI': 

In [77]:
processed_data_fname = "/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/processed_data/processed_data_135_tickers.csv"
all_df.to_csv(processed_data_fname, index=False)

In [79]:
processed_data_fname = "/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/processed_data/processed_data_135_tickers.csv"
df = pd.read_csv(processed_data_fname)